In [1]:
import json
import re
from functools import reduce
from copy import deepcopy

In [2]:
f = open('./data/cloze.ipynb', 'r')
nb = f.read()
nbd = json.loads(nb)

In [3]:
cells = [ c for c in nbd['cells'] ]
get_cells = lambda e: [c for c in cells if c['cell_type'] == e]

In [4]:
c_code = get_cells('code')

In [5]:
txt = c_code[0]['source']; txt

['%reload_ext autoreload\n', '%autoreload 2\n', '%matplotlib inline']

In [6]:
txt.append('cloze')

In [7]:
def isCloze(txtArr, cloze_marker=r'cloze', checkEmpty=False):
    present = [re.search(cloze_marker,s) for s in txtArr]
    match = None
    if len(present):
        match = reduce(lambda c, acc: c if c else acc, present)
    
    if match is not None:
        return True
    else:
        return False

isCloze(txt)

True

In [8]:
has_cloze = ['before\n', '# cloze {\n', 'code', '# }\n']
no_cloze = ['before\n', '# cl \n', 'code', '# } c\n']
no_cloze_empty = ['before\n', '# cl \n', 'code', '# } c\n', '']
has_cloze_empty = ['before\n', '# cl \n', 'code', '# } c\n', '']
empty = []

assert(isCloze(has_cloze) == True)
assert(isCloze(no_cloze) == False)
assert(isCloze(no_cloze_empty) == False)
assert(isCloze(has_cloze_empty) == False )
assert(isCloze(empty) == False )

In [9]:
c_code[14]

{'cell_type': 'code',
 'execution_count': 43,
 'metadata': {},
 'outputs': [],
 'source': ['class DeepNN(nn.Module):\n',
  '    ### cloze {\n',
  '    def __init__(self, layers):\n',
  '        super().__init__()\n',
  '        print(layers)\n',
  '        self.layers = {}\n',
  '        for i in range(len(layers) -1):\n',
  '            print(i, [layers[i], layers[i+1]])\n',
  '            self.layers[f"lin_{i}"] = nn.Linear(layers[i], layers[i+1])\n',
  '    def forward(self, xb):\n',
  '        ab = xb\n',
  '        for i in range(len(layers) -1):\n',
  '            ab = F.relu(self.layers[f"lin_{i}"](ab))\n',
  '        return ab\n',
  '    ### } cloze ']}

In [10]:
isCloze(c_code[14]['source'])

True

In [11]:
start_cloze_re = r'# cloze {'
end_cloze_re = r'# }'

# should return cloze code lines and the mutated source array

def extractCloze(cell_source):
    start_i = None
    end_i = None
    output = []
    for i, x in enumerate(cell_source):
        
        if isCloze([x], start_cloze_re):
            start_i = i 
        
        if start_i is not None and isCloze([x], end_cloze_re):
            end_i = i
            
            if not end_i - start_i == 1:
#                 print("EXTRACTING...")
                cloze_block = cell_source[start_i+1:end_i] # content of cloze tag
                sliced_source = cell_source[:start_i+1] + cell_source[end_i:] # includes cloze tag

                output.append((cloze_block, sliced_source))

            # reset detection
            start_i = None
            end_i = None
             
    
    return output

In [12]:
code_lines = [ f'code{i+1}\n' for i in range(10) ]
has_cloze = ['before\n', '# cloze {\n'] + code_lines  + ['# }\n']
deleted_cloze = ['before\n', '# cloze {\n'] + ['# }\n']

print(extractCloze(has_cloze)[0])
assert(extractCloze(has_cloze)[0][0] == code_lines)

print(extractCloze(deleted_cloze))
assert(extractCloze(deleted_cloze) == []) # assume you return nothing

(['code1\n', 'code2\n', 'code3\n', 'code4\n', 'code5\n', 'code6\n', 'code7\n', 'code8\n', 'code9\n', 'code10\n'], ['before\n', '# cloze {\n', '# }\n'])
[]


In [13]:
# if there are 2 clozes blocks within 1 cell
# num of cloze in one cell
num = 2

code_lines = lambda y: [ f'code{i+1}_{y}\n' for i in range(2) ]
cloze_list = [ ['# cloze {\n', *code_lines(i+1), '# }\n'] for i in range(num) ]
cloze_list_flat = [l for sublist in cloze_list for l in sublist]
has_multi_cloze = ['before\n', *cloze_list_flat, 'random end\n']


# print(extractCloze(has_multi_cloze)[1])
processed = extractCloze(has_multi_cloze)
for i in range(num):
    print(i)
    clz, cln_src = processed[i]
    assert(clz == code_lines(i+1))
    print(cln_src)

0
['before\n', '# cloze {\n', '# }\n', '# cloze {\n', 'code1_2\n', 'code2_2\n', '# }\n', 'random end\n']
1
['before\n', '# cloze {\n', 'code1_1\n', 'code2_1\n', '# }\n', '# cloze {\n', '# }\n', 'random end\n']


In [14]:
extractCloze(c_code[14]['source'])

[(['    def __init__(self, layers):\n',
   '        super().__init__()\n',
   '        print(layers)\n',
   '        self.layers = {}\n',
   '        for i in range(len(layers) -1):\n',
   '            print(i, [layers[i], layers[i+1]])\n',
   '            self.layers[f"lin_{i}"] = nn.Linear(layers[i], layers[i+1])\n',
   '    def forward(self, xb):\n',
   '        ab = xb\n',
   '        for i in range(len(layers) -1):\n',
   '            ab = F.relu(self.layers[f"lin_{i}"](ab))\n',
   '        return ab\n'],
  ['class DeepNN(nn.Module):\n', '    ### cloze {\n', '    ### } cloze '])]

## all together now

In [15]:
get_cells = lambda cells, e: [c for c in cells if c['cell_type'] == e]

def readNotebook(path):
    # read file
    f = open(path, 'r')
    nb = f.read()
    nbd = json.loads(nb)
    
    
    cloze_blocks = []
    
    for i,c in enumerate(nbd['cells']):
        # go inside 1 cell
        # extract all cloze blocks
        if c['cell_type'] == 'code' and isCloze(c['source']):
            processed = extractCloze(c['source'])
            
            # add multiple clozes inside one cell
            for data in processed:
                clz_src, cln_src = data
                cloze_blocks.append({"i":i, "clz":clz_src, "cln_src":cln_src})
            
    output = []
    
    print('cloze_blocks: ', len(cloze_blocks))
    
    for clz in cloze_blocks:
        new_code_cells = deepcopy([c for c in nbd['cells']])
        new_code_cells[clz['i']]['source']=clz["cln_src"]
        new_nb = deepcopy({k:nbd[k] for k in nbd})
        new_nb['cells'] = new_code_cells
        
        output.append((clz['clz'], new_nb))
    

    return output

In [16]:
x = readNotebook('./data/cloze.ipynb')

cloze_blocks:  1


In [17]:
# full e2e test!
# only one cloze in 1 cell of this notebook

def get_json(fpath):
    f = open(fpath, 'r')
    file = json.loads(f.read())
    f.close()
    return file

# some test data
expected_answer = get_json('./data/cloze_answer.json')['answer']
expected_extract_nb = get_json('./data/cloze_e.ipynb')

answer, extract_nb = readNotebook('./data/cloze.ipynb')[0]


print(answer)
assert(answer == expected_answer)
assert(extract_nb == expected_extract_nb)

cloze_blocks:  1
['    def __init__(self, layers):\n', '        super().__init__()\n', '        print(layers)\n', '        self.layers = {}\n', '        for i in range(len(layers) -1):\n', '            print(i, [layers[i], layers[i+1]])\n', '            self.layers[f"lin_{i}"] = nn.Linear(layers[i], layers[i+1])\n', '    def forward(self, xb):\n', '        ab = xb\n', '        for i in range(len(layers) -1):\n', '            ab = F.relu(self.layers[f"lin_{i}"](ab))\n', '        return ab\n']


## multi cloze in single notebook!

In [19]:
idxs = [i for i in range(3)]

expected_answers = get_json('./data/multi_answers.json')['answers']
expected_extract_nb = [ get_json(f"./data/multi_cloze-{i+1}.ipynb") for i in idxs]



output = readNotebook('./data/multi_cloze.ipynb')


for i in idxs:
    answer, extract_nb = output[i]
    print(i)
    
    # extracted answers are correct
    assert(answer == expected_answers[i])
    
    # extracted new nbs are correct
    for j,line in enumerate(expected_extract_nb[i]['cells'][17]['source']):

        if not extract_nb['cells'][17]['source'][j] == line:
            print(extract_nb['cells'][17]['source'][j],  l)
            raise Exception("line not equal")
        
    assert(extract_nb == expected_extract_nb[i])
    



cloze_blocks:  3
0
1
2
